In [1]:
import numpy as np
import os
from tqdm.auto import tqdm
# from pathos.multiprocessing import ProcessingPool as Pool
# from functools import partial
import pandas as pd
from PIL import Image

In [2]:
# clone Scaled_YOLOv4
!git clone https://github.com/tom-auger/ScaledYOLOv4.git
%cd ScaledYOLOv4/
#checkout the yolov4-large branch
!git checkout yolov4-large

Cloning into 'ScaledYOLOv4'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 170 (delta 28), reused 27 (delta 13), pack-reused 128
Receiving objects: 100% (170/170), 366.26 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/kaggle/working/ScaledYOLOv4
Branch 'yolov4-large' set up to track remote branch 'yolov4-large' from 'origin'.
Switched to a new branch 'yolov4-large'


In [3]:
df_train = pd.read_csv('/kaggle/input/vinbigstratifiedyolo/vinbigstratified/train_orig.csv')
df_val = pd.read_csv('/kaggle/input/vinbigstratifiedyolo/vinbigstratified/validation_orig.csv')
df_test = pd.read_csv('/kaggle/input/vinbigstratifiedyolo/vinbigstratified/test_orig.csv')

In [4]:
import torch
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.7.0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [5]:
#install mish activation funciton for cuda
%cd ..
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install
%cd ..

/kaggle/working
Cloning into 'mish-cuda'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 195 (delta 7), reused 79 (delta 3), pack-reused 107
Receiving objects: 100% (195/195), 208.77 KiB | 869.00 KiB/s, done.
Resolving deltas: 100% (56/56), done.
/kaggle/working/mish-cuda
/opt/conda/lib/python3.7/distutils/extension.py:131: UserWarning: Unknown Extension options: 'headers'
  warnings.warn(msg)
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/mish_cuda
copying src/mish_cuda/__init__.py -> build/lib.linux-x86_64-3.7/mish_cuda
running egg_info
creating src/mish_cuda.egg-info
writing src/mish_cuda.egg-info/PKG-INFO
writing dependency_links to src/mish_cuda.egg-info/dependency_links.txt
writing requirements to src/mish_cuda.egg-info/requires.txt
writing top-level names to src/mish_cuda.egg-info/top_level.txt
writing manif

In [6]:
%mkdir data
def write_to_file(imgs, dataset_name):
    with open(f"data/{dataset_name}_abnormsonly.txt", 'wt') as f:
        for img in imgs:
            f.write(f"/kaggle/input/vinbigstratifiedyolo/vinbigstratified/{dataset_name}/{img}.png\n")

train_imgs = df_train[df_train['class_id'] != 14]['image_id'].unique()
val_imgs = df_val[df_val['class_id'] != 14]['image_id'].unique()
test_imgs = df_test[df_test['class_id'] != 14]['image_id'].unique()

write_to_file(train_imgs, 'train')
write_to_file(val_imgs, 'validation')
write_to_file(test_imgs, 'test')

In [7]:
# abnormalities = ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']

# %mkdir data
# def write_to_file(imgs, dataset_name, ab_name):
#     with open(f"data/{dataset_name}_{ab_name}.txt", 'wt') as f:
#         for img in imgs:
#             f.write(f"/kaggle/input/mlpyoloall/{dataset_name}/{img}.png\n")
    
# def write_yaml_file(ab):
#     with open(f"data/{ab}.yaml", 'wt') as f:
#         f.write(f"""train: ../data/train_{ab}.txt
# val: ../data/validation_{ab}.txt

# nc: 14
# names: ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']
# """)
        
# for ab in abnormalities:
#     train_imgs = df_train[df_train['class_name'] == ab]['image_id'].unique()
#     val_imgs = df_val[df_val['class_name'] == ab]['image_id'].unique()
#     test_imgs = df_test[df_test['class_name'] == ab]['image_id'].unique()
    
#     # Write each list to a data spec file
#     ab = ab.replace('/', '_').replace(' ', '_')
#     write_to_file(train_imgs, 'train', ab)
#     write_to_file(val_imgs, 'validation', ab)
#     write_to_file(test_imgs, 'test', ab)
    
#     # Write the data yaml file
#     write_yaml_file(ab)

In [8]:
with open(f"data.yaml", 'wt') as f:
        f.write(f"""train: ../data/train_abnormsonly.txt
val: ../data/validation_abnormsonly.txt

nc: 14
names: ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']
""")

In [9]:
with open("hypparams.yaml", 'wt') as f:
    f.write("""# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials


lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
giou: 0.05  # GIoU loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.5  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mixup: 0.0  # image mixup (probability)
""")

In [10]:
%cd ScaledYOLOv4/

/kaggle/working/ScaledYOLOv4


In [11]:
!python train.py --labels 'labels_sup' --img 640 --batch 16 --epochs 40 --resume '/kaggle/input/mlpyolo-labels-sup/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/last_yolov4-csp-results.pt' --data '../data.yaml' --hyp '../hypparams.yaml' --cfg ./models/yolov4-csp.yaml --weights '' --name yolov4-csp-results --cache

Resuming training from /kaggle/input/mlpyolo-labels-sup/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/last_yolov4-csp-results.pt
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=True, cfg='./models/yolov4-csp.yaml', data='../data.yaml', device='', epochs=40, evolve=False, global_rank=-1, hyp='../hypparams.yaml', img_size=[640, 640], labels='labels_sup', local_rank=-1, logdir='runs/', multi_scale=False, name='yolov4-csp-results', noautoanchor=False, nosave=False, notest=False, onlyclass=-1, rect=False, resume='/kaggle/input/mlpyolo-labels-sup/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/last_yolov4-csp-results.pt', single_cls=False, sync_bn=False, total_batch_size=16, weights='/kaggle/input/mlpyolo-labels-sup/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/last_yolov4-csp-results.pt', world_size=1)
Start Tensorboard with "tensorboard --logdir runs/", view at http://localho